In [10]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [11]:
# read in data
data = pd.read_csv('data/en-annotated.tsv', sep='\t', header=None, names=['sentence', 'label_raw'])

# get only the first label for now
# unsure if that's the most "important" one or what to do later
data['label_raw'] = data['label_raw'].str.split(',')
data['label'] = data.label_raw.str[0]

# summarize first label
print(data.label.value_counts())
data.head()

1    3721
2    2706
8    2686
5    1845
4    1797
6    1704
3    1640
7    1429
Name: label, dtype: int64


,sentence,label_raw,label
0,", ...",[1],1
1,!,"[1, 4, 7]",1
2,... And I don't think we need to discuss the T...,"[8, 1]",8
3,* So get up out of your bed,[1],1
4,A confession that you hired [PERSON] ... and a...,"[1, 6]",1


In [14]:
train_in, test_in, train_labels, test_labels = train_test_split(data['sentence'], data['label'], test_size = 0.33)
print('Train dataset shape: ', train_in.shape)
print('Test dataset shape: ', test_in.shape)

Train dataset shape:  (11743,)
Test dataset shape:  (5785,)
